In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn import preprocessing

In [2]:
data = pd.read_csv("source/train.csv")

In [3]:
data

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.50,Premium,F,VS2,61.5,58.0,7.32,7.34,4.51,9.588
1,1,2.01,Very Good,E,SI2,60.6,59.0,8.11,8.25,4.96,9.748
2,2,0.50,Ideal,E,SI1,61.6,57.0,5.13,5.09,3.15,7.255
3,3,0.25,Very Good,F,VVS2,61.6,57.0,4.05,4.08,2.50,6.450
4,4,0.52,Ideal,G,VS2,62.0,55.0,5.16,5.19,3.21,7.721
...,...,...,...,...,...,...,...,...,...,...,...
40450,40450,1.04,Very Good,I,SI2,59.6,57.0,6.60,6.62,3.94,8.190
40451,40451,0.51,Very Good,E,SI1,63.3,59.0,5.09,5.05,3.21,7.246
40452,40452,1.51,Ideal,H,VS2,62.6,57.0,7.37,7.33,4.60,9.277
40453,40453,2.02,Premium,H,VS2,61.3,60.0,8.16,8.11,4.99,9.680


In [4]:
data.corr()

,id,carat,depth,table,x,y,z,price
id,1.000000,-0.006196,0.001098,-0.008829,-0.007112,-0.006420,-0.007766,-0.006926
carat,-0.006196,1.000000,0.025089,0.182590,0.974516,0.950563,0.947930,0.920878
depth,0.001098,0.025089,1.000000,-0.300152,-0.028389,-0.032561,0.091421,-0.000924
table,-0.008829,0.182590,-0.300152,1.000000,0.196031,0.184322,0.150142,0.158128
x,-0.007112,0.974516,-0.028389,0.196031,1.000000,0.973617,0.965597,0.957653
y,-0.006420,0.950563,-0.032561,0.184322,0.973617,1.000000,0.945958,0.934700
z,-0.007766,0.947930,0.091421,0.150142,0.965597,0.945958,1.000000,0.930157
price,-0.006926,0.920878,-0.000924,0.158128,0.957653,0.934700,0.930157,1.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       40455 non-null  int64  
 1   carat    40455 non-null  float64
 2   cut      40455 non-null  object 
 3   color    40455 non-null  object 
 4   clarity  40455 non-null  object 
 5   depth    40455 non-null  float64
 6   table    40455 non-null  float64
 7   x        40455 non-null  float64
 8   y        40455 non-null  float64
 9   z        40455 non-null  float64
 10  price    40455 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 3.4+ MB


In [6]:
print(data.cut.unique())
print(data.color.unique())
print(data.clarity.unique())

['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
['F' 'E' 'G' 'D' 'J' 'I' 'H']
['VS2' 'SI2' 'SI1' 'VVS2' 'VS1' 'VVS1' 'IF' 'I1']


#### En vez de hacer dummies ,  vamos a ver si estas variables float tienen un orden de importancia comparandolas con el precio.

In [7]:
data["price/carat"]= data["price"] / data["carat"]

In [8]:
print(data.groupby('cut')['price/carat'].mean().sort_values())
print(data.groupby('color')['price/carat'].mean().sort_values())
print(data.groupby('clarity')['price/carat'].mean().sort_values())

cut
Fair          9.284466
Good         11.663682
Premium      11.807049
Very Good    12.446719
Ideal        13.893712
Name: price/carat, dtype: float64
color
J     9.017189
I    10.422280
H    11.510122
G    12.992913
F    13.141893
D    14.133644
E    14.300001
Name: price/carat, dtype: float64
clarity
I1       7.466673
SI2      9.212076
SI1     11.440938
VS2     13.008773
VS1     13.512145
VVS2    15.815193
VVS1    17.459082
IF      17.667809
Name: price/carat, dtype: float64


In [9]:
data['cut']=data['cut'].replace({'Ideal':1,'Good':2,'Very Good':3,'Fair':4,'Premium':5})
data['color']=data['color'].replace({'E':1,'D':2,'F':3,'G':4,'H':5,'I':6,'J':7})
data['clarity']=data['clarity'].replace({'VVS1':1,'IF':2,'VVS2':3,'VS1':4,'I1':5,'VS2':6,'SI1':7,'SI2':8})

In [10]:
data = data.drop(["price/carat"], axis=1)

In [11]:
data.corr()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
id,1.000000,-0.006196,-0.010850,0.002469,-0.002531,0.001098,-0.008829,-0.007112,-0.006420,-0.007766,-0.006926
carat,-0.006196,1.000000,0.163227,0.289227,0.320217,0.025089,0.182590,0.974516,0.950563,0.947930,0.920878
cut,-0.010850,0.163227,1.000000,0.040410,0.164484,-0.087940,0.499487,0.167382,0.150368,0.145017,0.123236
color,0.002469,0.289227,0.040410,1.000000,-0.023420,0.044665,0.024083,0.269491,0.263042,0.265130,0.159381
clarity,-0.002531,0.320217,0.164484,-0.023420,1.000000,0.044650,0.149458,0.343427,0.331780,0.335356,0.206736
depth,0.001098,0.025089,-0.087940,0.044665,0.044650,1.000000,-0.300152,-0.028389,-0.032561,0.091421,-0.000924
table,-0.008829,0.182590,0.499487,0.024083,0.149458,-0.300152,1.000000,0.196031,0.184322,0.150142,0.158128
x,-0.007112,0.974516,0.167382,0.269491,0.343427,-0.028389,0.196031,1.000000,0.973617,0.965597,0.957653
y,-0.006420,0.950563,0.150368,0.263042,0.331780,-0.032561,0.184322,0.973617,1.000000,0.945958,0.934700
z,-0.007766,0.947930,0.145017,0.265130,0.335356,0.091421,0.150142,0.965597,0.945958,1.000000,0.930157


In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.10" 2021-01-19 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.10+8-LTS-162); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.10+8-LTS-162, mixed mode)
  Starting server from /usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/jb/5q85jkg95p1cnnjq2x9by0nm0000gn/T/tmpai06us8i
  JVM stdout: /var/folders/jb/5q85jkg95p1cnnjq2x9by0nm0000gn/T/tmpai06us8i/h2o_a_m_started_from_python.out
  JVM stderr: /var/folders/jb/5q85jkg95p1cnnjq2x9by0nm0000gn/T/tmpai06us8i/h2o_a_m_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_a_m_q2qf1y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [13]:
## Diamonds TEST

In [14]:
data_test = pd.read_csv("source/test.csv")

In [15]:
data_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.51,Very Good,E,SI2,58.3,61.9,5.19,5.20,3.04
1,1,1.02,Very Good,F,SI2,63.0,58.0,6.37,6.43,4.03
2,2,0.59,Ideal,D,VS2,61.9,56.0,5.39,5.34,3.32
3,3,0.90,Very Good,D,VS2,62.3,56.0,6.14,6.18,3.84
4,4,2.01,Premium,J,VS1,60.2,61.0,8.23,8.16,4.93


In [16]:
data_test['cut']=data_test['cut'].replace({'Ideal':1,'Good':2,'Very Good':3,'Fair':4,'Premium':5})
data_test['color']=data_test['color'].replace({'E':1,'D':2,'F':3,'G':4,'H':5,'I':6,'J':7})
data_test['clarity']=data_test['clarity'].replace({'VVS1':1,'IF':2,'VVS2':3,'VS1':4,'I1':5,'VS2':6,'SI1':7,'SI2':8})

In [17]:
data_test = data_test.drop(columns = ["id"])

In [18]:
h2train = h2o.H2OFrame(data)
h2pred = h2o.H2OFrame(data_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [19]:
data.columns

Index(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y',
       'z', 'price'],
      dtype='object')

In [20]:
X = list(data.drop(columns = ['price', 'id'] ).columns)
y = 'price'

In [21]:
automl = H2OAutoML(max_runtime_secs = 2500)
automl.train(X,y,training_frame = h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [22]:
leader_board = automl.leaderboard
leader_board.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210228_205354,0.00815085,0.090282,0.00815085,0.0653878,0.0102812
StackedEnsemble_BestOfFamily_AutoML_20210228_205354,0.00817735,0.0904287,0.00817735,0.065549,0.0103001
XGBoost_grid__1_AutoML_20210228_205354_model_1,0.00819437,0.0905228,0.00819437,0.0655716,0.0103103
XGBoost_grid__1_AutoML_20210228_205354_model_3,0.00833359,0.0912885,0.00833359,0.0649224,0.0103405
GBM_1_AutoML_20210228_205354,0.00883634,0.0940018,0.00883634,0.0692449,0.0108167
XGBoost_3_AutoML_20210228_205354,0.008899,0.0943345,0.008899,0.0688274,0.0107825
GBM_3_AutoML_20210228_205354,0.00892083,0.0944501,0.00892083,0.0692803,0.0108429
XGBoost_2_AutoML_20210228_205354,0.00896523,0.0946849,0.00896523,0.0681364,0.0107448
GBM_2_AutoML_20210228_205354,0.00900275,0.0948828,0.00900275,0.0696564,0.0109228
XGBoost_grid__1_AutoML_20210228_205354_model_2,0.00914178,0.0956127,0.00914178,0.06877,0.0108743


In [ ]:
predictions = automl.leader.predict(h2pred)
predictions.head()

In [ ]:
sub = predictions.as_data_frame()
columns = ['id','price']
sub =sub.reset_index()
sub.columns = columns



In [ ]:
sub.to_csv('h2o2.csv',index=False, header = True )